In [1]:
import pandas as pd
import docx
import csv
import re
from datetime import datetime, timedelta
from typing import List, Dict, Optional, Tuple
import logging

# Настройка логирования
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
logger = logging.getLogger(__name__)

class CourseScheduleProcessor:
    """Класс для полной обработки расписания курсов"""
    
    def __init__(self):
        # Компилируем regex один раз
        self.group_pattern = re.compile(r'[Гг]руппа\s+(\d+)')
        self.lesson_cleanup_patterns = [
            re.compile(r'\s+\-\s+занятие\s+\d+.*'),
            re.compile(r'\s+группа\s+\d+.*')
        ]
        
        # Словарь дней недели
        self.weekdays = {
            0: "Понедельник", 1: "Вторник", 2: "Среда", 3: "Четверг",
            4: "Пятница", 5: "Суббота", 6: "Воскресенье"
        }
        
        # Колонки финального файла
        self.final_columns = [
            "№", "Название мероприятия", "Код мероприятия", "Код типового мероприятия",
            "Тип мероприятия", "Код программы мероприятий", "Дата начала обучения",
            "Дата окончания обучения", "Категория обучения", "Контрагент", "Разъяснения",
            "Формат обучения (для программ)", "Академические часы", "Вид обучения",
            "Место проведения обучения", "Типы выдаваемых документов", "Номер контракта",
            "Дата заключения контракта", "СПУ", "Ссылки на курс"
        ]
    
    def extract_table_from_docx(self, docx_path: str, csv_path: str) -> bool:
        """Извлекает таблицу из DOCX и сохраняет в CSV"""
        try:
            doc = docx.Document(docx_path)
            if not doc.tables:
                logger.error("В документе нет таблиц")
                return False
            
            with open(csv_path, 'w', newline='', encoding='utf-8') as csv_file:
                writer = csv.writer(csv_file)
                for row in doc.tables[0].rows:
                    writer.writerow([cell.text.strip() for cell in row.cells])
            
            logger.info(f"Таблица извлечена в {csv_path}")
            return True
        except Exception as e:
            logger.error(f"Ошибка извлечения таблицы: {e}")
            return False
    
    def read_courses_csv(self, file_path: str) -> List[Dict]:
        """Читает CSV с курсами и нормализует заголовки"""
        try:
            # Определяем разделитель
            with open(file_path, 'r', encoding='utf-8') as f:
                sample = f.read(1024)
                delimiter = ";" if sample.count(";") > sample.count(",") else ","
            
            df = pd.read_csv(file_path, encoding='utf-8', sep=delimiter)
            
            # Маппинг заголовков
            header_mapping = self._create_header_mapping(df.columns.tolist())
            
            courses = []
            for _, row in df.iterrows():
                if not any(row.values):  # Пропускаем пустые строки
                    continue
                
                course_data = {}
                for expected, actual in header_mapping.items():
                    course_data[expected] = str(row.get(actual, "")) if pd.notna(row.get(actual)) else ""
                
                # Значения по умолчанию
                if not course_data.get("График"):
                    course_data["График"] = "Ежедневно"
                if not course_data.get("Время-занятий"):
                    course_data["Время-занятий"] = "День"
                
                courses.append(course_data)
            
            logger.info(f"Прочитано {len(courses)} курсов")
            return courses
            
        except Exception as e:
            logger.error(f"Ошибка чтения CSV: {e}")
            return self._create_test_data()
    
    def _create_header_mapping(self, headers: List[str]) -> Dict[str, str]:
        """Создает маппинг заголовков"""
        expected = [
            "Название программы", "Код мероприятия", "Объем академ. часов",
            "Время-занятий", "Дата начала", "Дата окончания",
            "Время начала", "Время окончания", "График", "Формат обучения"
        ]
        
        mapping = {}
        # Прямое сопоставление
        for exp in expected:
            if exp in headers:
                mapping[exp] = exp
        
        # Нечеткое сопоставление
        for exp in expected:
            if exp not in mapping:
                for header in headers:
                    if self._fuzzy_match(exp, header):
                        mapping[exp] = header
                        break
        
        # Сопоставление по позиции как fallback
        for i, exp in enumerate(expected):
            if exp not in mapping and i < len(headers):
                mapping[exp] = headers[i]
        
        return mapping
    
    def _fuzzy_match(self, expected: str, actual: str) -> bool:
        """Нечеткое сопоставление заголовков"""
        expected_lower = expected.lower()
        actual_lower = actual.lower()
        
        fuzzy_rules = {
            "дата начала": ["дата", "начал"],
            "дата окончания": ["дата", "окон", "конец"],
            "время начала": ["время", "начал"],
            "время окончания": ["время", "окон", "конец"],
            "формат обучения": ["формат"],
            "график": ["график", "период"]
        }
        
        for pattern, keywords in fuzzy_rules.items():
            if pattern in expected_lower:
                return all(kw in actual_lower for kw in keywords)
        
        return False
    
    def generate_detailed_schedule(self, courses: List[Dict]) -> List[Dict]:
        """Генерирует детализированное расписание"""
        detailed_schedule = []
        processed_courses = self._process_course_groups(courses)
        
        for course in processed_courses:
            # Основная запись программы
            main_entry = self._create_main_entry(course)
            detailed_schedule.append(main_entry)
            
            # Генерируем занятия
            lessons = self._generate_lessons(course)
            detailed_schedule.extend(lessons)
        
        return detailed_schedule
    
    def _process_course_groups(self, courses: List[Dict]) -> List[Dict]:
        """Обрабатывает группы курсов"""
        processed = []
        current_program = None
        group_counters = {}
        
        for course in courses:
            program_name = course["Название программы"].strip()
            
            if program_name:
                current_program = program_name
                group_counters[current_program] = group_counters.get(current_program, 0) + 1
                course["Название программы"] = f"{program_name} Группа {group_counters[current_program]}"
            elif current_program:
                group_counters[current_program] += 1
                course["Название программы"] = f"{current_program} Группа {group_counters[current_program]}"
            
            processed.append(course)
        
        return processed
    
    def _create_main_entry(self, course: Dict) -> Dict:
        """Создает основную запись программы"""
        return {
            "Название мероприятия": course["Название программы"],
            "Код": course["Код мероприятия"],
            "Дата начала обучения": course["Дата начала"],
            "Дата окончания обучения": course["Дата окончания"],
            "Время начала": course["Время начала"],
            "Время окончания": course["Время окончания"],
            "Академические часы": course["Объем академ. часов"],
            "Формат обучения": course.get("Формат обучения", ""),
            "Примечания": self._normalize_schedule(course["График"])
        }
    
    def _generate_lessons(self, course: Dict) -> List[Dict]:
        """Генерирует занятия для курса"""
        lessons = []
        start_date = datetime.strptime(course["Дата начала"], "%d.%m.%Y")
        end_date = datetime.strptime(course["Дата окончания"], "%d.%m.%Y")
        schedule = course["График"]
        
        current_date = start_date
        lesson_number = 1
        
        while current_date <= end_date:
            if self._is_day_in_schedule(current_date, schedule):
                lesson = {
                    "Название мероприятия": f"{course['Название программы']} - Занятие {lesson_number}",
                    "Код": course["Код мероприятия"],
                    "Дата начала обучения": current_date.strftime("%d.%m.%Y"),
                    "Дата окончания обучения": current_date.strftime("%d.%m.%Y"),
                    "Время начала": course["Время начала"],
                    "Время окончания": course["Время окончания"],
                    "Академические часы": "",
                    "Формат обучения": "",
                    "Примечания": self.weekdays[current_date.weekday()]
                }
                lessons.append(lesson)
                lesson_number += 1
            
            current_date += timedelta(days=1)
        
        return lessons
    
    def _is_day_in_schedule(self, date_obj: datetime, schedule: str) -> bool:
        """Проверяет, входит ли день в расписание"""
        day_name = self.weekdays[date_obj.weekday()]
        
        if "ежедневно" in schedule.lower():
            return True
        
        return day_name in schedule
    
    def _normalize_schedule(self, schedule: str) -> str:
        """Нормализует расписание"""
        if not schedule or "ежедневно" in schedule.lower():
            return ", ".join(self.weekdays.values())
        
        days = [day.strip() for day in schedule.split(",")]
        valid_days = []
        
        for day in days:
            for valid_day in self.weekdays.values():
                if valid_day.lower() in day.lower():
                    valid_days.append(valid_day)
                    break
        
        return ", ".join(valid_days) if valid_days else ", ".join(self.weekdays.values())
    
    def add_datetime_and_hours(self, df: pd.DataFrame) -> pd.DataFrame:
        """Добавляет полные даты и рассчитывает академические часы"""
        # Создаем datetime столбцы
        df['Дата и время начала'] = df.apply(
            lambda row: self._combine_date_time(row['Дата начала обучения'], row['Время начала']),
            axis=1
        )
        df['Дата и время окончания'] = df.apply(
            lambda row: self._combine_date_time(row['Дата окончания обучения'], row['Время окончания']),
            axis=1
        )
        
        # Рассчитываем академические часы для занятий
        df['Академические часы'] = df['Академические часы'].fillna(0).astype(str)
        
        for i, row in df.iterrows():
            if " - Занятие " in str(row["Название мероприятия"]):
                hours = self._calculate_academic_hours(
                    row['Дата и время начала'], 
                    row['Дата и время окончания']
                )
                df.at[i, 'Академические часы'] = str(hours)
        
        return df
    
    def _combine_date_time(self, date_str: str, time_str: str) -> Optional[str]:
        """Объединяет дату и время"""
        try:
            if pd.isna(date_str) or pd.isna(time_str):
                return None
            
            date_parts = str(date_str).split('.')
            time_parts = str(time_str).split(':')
            
            if len(date_parts) != 3 or len(time_parts) != 2:
                return None
            
            day, month, year = map(int, date_parts)
            hour, minute = map(int, time_parts)
            
            dt = datetime(year, month, day, hour, minute)
            return dt.strftime('%d.%m.%Y %H:%M')
        except:
            return None
    
    def _calculate_academic_hours(self, start_str: str, end_str: str) -> int:
        """Рассчитывает академические часы"""
        try:
            if not start_str or not end_str:
                return 0
            
            start_dt = datetime.strptime(start_str, '%d.%m.%Y %H:%M')
            end_dt = datetime.strptime(end_str, '%d.%m.%Y %H:%M')
            
            diff_minutes = (end_dt - start_dt).total_seconds() / 60
            return round(diff_minutes / 45)  # 1 ак. час = 45 минут
        except:
            return 0
    
    def add_codes_and_types(self, df: pd.DataFrame) -> pd.DataFrame:
        """Добавляет коды мероприятий и типы"""
        # Добавляем коды мероприятий
        df['Код мероприятия'] = df.apply(
            lambda row: self._generate_event_code(
                row['Название мероприятия'], 
                row['Дата начала обучения']
            ), axis=1
        )
        
        # Добавляем типы мероприятий
        df['Тип мероприятия'] = df['Название мероприятия'].apply(
            lambda name: "Дистанционное мероприятие" if "Занятие" in str(name) 
            else "Программа мероприятий"
        )
        
        # Обновляем формат обучения
        df['Формат обучения'] = df['Формат обучения'].astype(str)
        df.loc[df['Тип мероприятия'] == 'Программа мероприятий', 'Формат обучения'] = 'Дистанционная'
        
        # Добавляем коды программ
        df['Код программы мероприятий'] = self._add_program_codes(df)
        
        return df
    
    def _generate_event_code(self, event_name: str, start_date: str) -> str:
        """Генерирует код мероприятия"""
        try:
            date_parts = str(start_date).split('.')
            if len(date_parts) != 3:
                return 'invalid_date'
            
            day, month, year = date_parts
            date_code = f"{day}{month}{year[-2:]}"
            
            # Укорачиваем название
            words = str(event_name).split()[:5]
            name_code = '_'.join(words).replace(' ', '_')
            
            # Извлекаем номер группы
            group_match = self.group_pattern.search(str(event_name))
            group_suffix = f"_{group_match.group(1)}" if group_match else ""
            
            prefix = "H_" if "Занятие" in str(event_name) else "tp_H_"
            return f"{prefix}{name_code}_{date_code}{group_suffix}"
        except:
            return "error_code"
    
    def _add_program_codes(self, df: pd.DataFrame) -> List[str]:
        """Добавляет коды программ мероприятий"""
        codes = [""] * len(df)
        last_group_code = None
        
        for i, row in df.iterrows():
            name = str(row['Название мероприятия'])
            code = str(row['Код мероприятия'])
            
            if "Занятие" not in name and code.startswith("tp_H_"):
                last_group_code = code
            elif "Занятие" in name and last_group_code:
                codes[i] = last_group_code
        
        return codes
    
    def finalize_dataframe(self, df: pd.DataFrame) -> pd.DataFrame:
        """Финализирует DataFrame с нужными колонками и порядком"""
        # Удаляем ненужные колонки
        drop_cols = ['Дата начала обучения', 'Дата окончания обучения', 'Время начала', 'Время окончания']
        df = df.drop([col for col in drop_cols if col in df.columns], axis=1)
        
        # Переименовываем колонки
        rename_map = {
            "Код": "Код типового мероприятия",
            "Формат обучения": "Формат обучения (для программ)",
            "Дата и время начала": "Дата начала обучения",
            "Дата и время окончания": "Дата окончания обучения"
        }
        df = df.rename(columns=rename_map)
        
        # Добавляем недостающие колонки
        for col in self.final_columns:
            if col not in df.columns:
                df[col] = ""
        
        # Переупорядочиваем колонки
        return df[self.final_columns]
    
    def _create_test_data(self) -> List[Dict]:
        """Создает тестовые данные"""
        return [
            {
                "Название программы": "Python Базовый курс",
                "Код мероприятия": "PY-1",
                "Объем академ. часов": "24",
                "Дата начала": "21.01.2025",
                "Дата окончания": "28.01.2025",
                "Время начала": "12:00",
                "Время окончания": "15:00",
                "Формат обучения": "Дистанционная",
                "График": "Ежедневно",
                "Время-занятий": "День"
            }
        ]
    
    def process_full_pipeline(self, docx_path: str, output_path: str = "1.reordered_file.csv") -> bool:
        """Полный пайплайн обработки"""
        try:
            logger.info("Начало обработки полного пайплайна")
            
            # 1. Извлечение таблицы из DOCX
            csv_temp = "1.courses_data.csv"
            if not self.extract_table_from_docx(docx_path, csv_temp):
                logger.warning("Используем тестовые данные")
                courses = self._create_test_data()
            else:
                # 2. Чтение данных курсов
                courses = self.read_courses_csv(csv_temp)
            
            # 3. Генерация детализированного расписания
            logger.info("Генерация расписания...")
            detailed_schedule = self.generate_detailed_schedule(courses)
            
            # 4. Создание DataFrame
            df = pd.DataFrame(detailed_schedule)
            
            # 5. Добавление datetime и академических часов
            logger.info("Добавление дат и часов...")
            df = self.add_datetime_and_hours(df)
            
            # 6. Добавление кодов и типов
            logger.info("Добавление кодов и типов...")
            df = self.add_codes_and_types(df)
            
            # 7. Финализация
            logger.info("Финализация...")
            df = self.finalize_dataframe(df)
            
            # 8. Сохранение
            df.to_csv(output_path, index=False, encoding='utf-8-sig', sep=';')
            logger.info(f"Файл сохранен: {output_path}")
            
            return True
            
        except Exception as e:
            logger.error(f"Ошибка в пайплайне: {e}")
            return False

def main():
    """Основная функция"""
    processor = CourseScheduleProcessor()
    
    # Обработка полного пайплайна
    docx_file = "test.docx"
    success = processor.process_full_pipeline(docx_file)
    
    if success:
        print("✅ Обработка завершена успешно!")
    else:
        print("❌ Ошибка при обработке")

if __name__ == "__main__":
    main()

INFO: Начало обработки полного пайплайна


ERROR: Ошибка извлечения таблицы: Package not found at 'test.docx'


INFO: Генерация расписания...


INFO: Добавление дат и часов...


INFO: Добавление кодов и типов...


INFO: Финализация...


INFO: Файл сохранен: 1.reordered_file.csv


✅ Обработка завершена успешно!
